# ToF demo

This Jupyter notebook invokes the Vespucci jobcontrol-api to

- Train the model for ToF
- Generate the network `.c`/`.h` sources
- Build the application firmware

### Initial setup

Dependencies are managed via `poetry`. You may install it with `pip install poetry`

Then, install all dependencies simply by invoking `poetry install` in this directory. This will create a virtualenvironment and place it in this folder as `.venv`.

Start a jupyter kernel in that environment and you're ready to run the notebook.

You will also need API credentials for both Vespucci and the stm32ai developer cloud. Devcloud accepts vespucci tokens from all environments; his examples runs on vespucci-dev, so you should use a token generated for that environment.


In [ ]:
# Configuration preamble
import json
import httpx
import config
import utils

client = httpx.Client(timeout=300, headers=utils.headers)


## Model training

This step will produce the `.h5` model artifact.

In [ ]:
with open("dataset", "rb") as dataset_file:
    print("Creating training job")
    job = client.post(
        config.JOBCONTROL_API_JOBS_ENDPOINT,
        files={
            "uploadedFile": dataset_file
        },
        data={
            "templateId": config.TRAINING_JOB_TEMPLATE_ID,
            "runtimeInput": json.dumps(
                {
                    "config": {
                        "dataset": {
                            "class_names": [
                                "One",
                                "Two",
                                "Three",
                                "Four"
                            ]
                        },
                        # "stm32ai": {
                        #     "version": "8.1.0"
                        # }
                    },
                }
            )
        }
    ).raise_for_status().json()


print(f"Created training job: {job!r}")

job_id = job["id"]

print("Blocking until job is done...")
job = await utils.block_until_done(job_id)

print("Training job is done. Downloading artifacts")
artifacts = utils.download_artifacts(job)

for key in artifacts:
    print(f"Writing {key}")
    with open(key, "wb") as file:
        file.write(artifacts[key])

# Ok for GC
del artifacts
client.delete(f"{config.JOBCONTROL_API_JOBS_ENDPOINT}/{job_id}")



## CubeAI

This step will generate `.c`/`.h` NN sources by invoking the STM32 Developer cloud APIs.

We'll download a zip containing said sources as well as the CubeAI runtime that will eventually be linked into the application firmware. 

In [ ]:

with open("best_model", "rb") as model_file:
    model = utils.NamedBuffer("model", model_file.read())

# devcloud API command
command = {
    "command": "generate",
    "arguments":{
        "options": {
            "includeLibraryForSerie": "M4",
            "includeLibraryForIde": "gcc",
            "allocateInputs": True,
            "allocateOutputs": True,
            "compression": "none",
            "optimization": "balanced"
        },
    },
    "version": "7.3.0"
}

print("Creating CubeAI job")
job = client.post(
    config.JOBCONTROL_API_JOBS_ENDPOINT,
    files={
        "model": model,
    },
    data={
        "templateId": config.CUBEAI_JOB_TEMPLATE_ID,
        "runtimeInput": json.dumps({
            "command": command
        })
    }
).raise_for_status().json()

print(f"Created CubeAI job: {job!r}")

job_id = job["id"]

print("Blocking until job is done...")
job = await utils.block_until_done(job_id)

print("CubeAI job is done. Downloading artifacts")
artifacts = utils.download_artifacts(job)

for key in artifacts:
    print(f"Writing {key}")
    with open(key, "wb") as file:
        file.write(artifacts[key])

# Ok for GC
del artifacts
client.delete(f"{config.JOBCONTROL_API_JOBS_ENDPOINT}/{job_id}")

## Building

In this step we build the output NN sources into a deployment-ready application firmware.

In [ ]:
with open("output", "rb") as cubeai_output_file:
    cubeai_output = utils.NamedBuffer("network", cubeai_output_file.read())

print("Creating builder job")
job = client.post(
    config.JOBCONTROL_API_JOBS_ENDPOINT,
    files={
        "network": cubeai_output
    },
    data={
        "templateId": config.COMPILATION_JOB_TEMPLATE_ID,
        "runtimeInput": json.dumps({
            "config": {
                "dataset": {
                    "class_names": [
                        "One",
                        "Two",
                        "Three",
                        "Four"
                    ]
                },
                # "stm32ai": {
                #     "version": "8.1.0"
                # }
            },
        })
    }
).raise_for_status().json()

print(f"Created builder job: {job!r}")

job_id = job["id"]

print("Blocking until job is done...")
job = await utils.block_until_done(job_id)

print("Builder job is done. Downloading artifacts")
artifacts = utils.download_artifacts(job)

for key in artifacts:
    print(f"Writing {key}")
    with open(key, "wb") as file:
        file.write(artifacts[key])

# Ok for GC
del artifacts
client.delete(f"{config.JOBCONTROL_API_JOBS_ENDPOINT}/{job_id}")

In [ ]:
# Cleanup

client.close()